In [65]:
#IMPORT DATA
import pandas as pd
import numpy as np
data_train = pd.read_csv('train.csv')
data_test=pd.read_csv('test.csv')

In [66]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [67]:
data_train.head()

,Retained,ID_SALES,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,0,CC139A,other,air,East Coast,PK,Middle,PK->Middle,C,other,...,0.570244,0.119431,2708.000000,0.888166,0.000000,0.000000,31.333333,3,2,190.000000
1,1,CC1600A,other,air,East Coast,K,Middle,K->Middle,C,other,...,0.510360,0.190362,1250.605592,1.000000,0.000000,0.000000,0.000000,0,1,199.424613
2,1,CC1836A,other,air,East Coast,PK,Middle,PK->Middle,C,other,...,0.623850,0.274595,2518.000000,0.929941,0.966667,0.400000,34.333333,3,-13,164.000000
3,1,CC242A,other,air,East Coast,PK,Middle,PK->Middle,C,other,...,0.592224,0.262095,2505.000000,0.894531,0.933333,0.733333,57.666667,3,-1,162.000000
4,0,CC2531A,other,air,East Coast,PK,Middle,PK->Middle,C,other,...,0.571137,0.088141,2569.000000,0.850497,0.000000,0.200000,54.333333,3,-4,195.000000


In [68]:
data_test.head()

,Retained,ID_SALES,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,1,CC1387A,other,air,East Coast,K,High,K->High,C,other,...,0.345746,0.103222,2167.0,0.991415,0.766667,0.829167,62.000000,3,-2,320.0
1,1,CC1701A,other,air,East Coast,PK,Middle,PK->Middle,C,other,...,0.404058,0.104028,2148.0,0.915161,1.000000,0.533333,44.666667,3,4,213.0
2,1,CC1773A,other,air,East Coast,K,Middle,K->Middle,C,other,...,0.517738,0.000000,1989.0,0.866585,1.000000,1.000000,23.000000,3,1,139.0
3,0,CC1830A,other,air,East Coast,Middle,Middle,Middle->Middle,C,other,...,0.721320,0.381479,2388.0,0.887128,0.000000,0.000000,50.000000,3,-12,399.0
4,1,CC2637A,other,air,East Coast,K,Middle,K->Middle,C,other,...,0.492463,0.168508,2360.0,0.957977,0.700000,0.800000,23.000000,4,-4,207.0


In [69]:
#basic statistic details about the data
data_train.describe(include="all")

,Retained,ID_SALES,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
count,3229.000000,3229,3229,3229,3229,3229,3229,3229,3229,3229,...,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000,3229.000000
unique,NaN,3137,5,2,4,6,4,13,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,SC1171A,CN|HD,air,East Coast,Middle,Middle,Middle->Middle,H,B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,2,1909,2661,2682,1666,2402,1465,2753,1298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.611644,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.567126,0.209859,1593.159208,0.901182,0.059056,0.346663,48.949030,3.054196,-0.176835,195.345928
std,0.487452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.232677,0.161110,669.146409,0.056090,0.284144,0.267200,14.273453,0.257186,5.836794,49.331680
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,73.000000,0.608229,-1.000000,0.000000,0.000000,0.000000,-43.000000,-149.000000
25%,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.451581,0.100512,1070.000000,0.873556,-0.016667,0.066667,39.333333,3.000000,-2.000000,162.000000
50%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.595420,0.192580,1679.000000,0.907843,0.000000,0.333333,49.666667,3.000000,0.000000,197.000000
75%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.724460,0.296565,2055.000000,0.937686,0.200000,0.559722,59.333333,3.000000,2.000000,225.000000


In [70]:
#Separate X and y (explanatory variables and target variable in train set)
X_train = data_train.iloc[:,2:]
column_names = list(X_train) 
Y_train = data_train.iloc[:,0] 
del X_train['Retained_class']
X_train.head()

,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,Region,CRM_Segment,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.570244,0.119431,2708.000000,0.888166,0.000000,0.000000,31.333333,3,2,190.000000
1,other,air,East Coast,K,Middle,K->Middle,C,other,other,other,...,0.510360,0.190362,1250.605592,1.000000,0.000000,0.000000,0.000000,0,1,199.424613
2,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.623850,0.274595,2518.000000,0.929941,0.966667,0.400000,34.333333,3,-13,164.000000
3,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.592224,0.262095,2505.000000,0.894531,0.933333,0.733333,57.666667,3,-1,162.000000
4,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.571137,0.088141,2569.000000,0.850497,0.000000,0.200000,54.333333,3,-4,195.000000


In [71]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Retained, dtype: int64

In [72]:
X_train_num = X_train.iloc[:,31:]
X_train_num.head()

,Days,Total_Pax,FPP_to_School_enrollment,MDR_Low_Grade,MDR_High_Grade,Total_School_Enrollment,NumberOfMeetingswithParents,DifferenceTraveltoFirstMeeting,DifferenceTraveltoLastMeeting,FRP_Cancelled,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,18,22,0.173405,-1.000000,8.000000,102.000000,1.000000,238.000000,244.000000,3.000000,0.570244,0.119431,2708.000000,0.888166,0.000000,0.000000,31.333333,3,2,190.000000
1,13,7,0.108986,2.137317,7.639715,420.230593,1.012401,259.413772,231.336913,3.152003,0.510360,0.190362,1250.605592,1.000000,0.000000,0.000000,0.000000,0,1,199.424613
2,33,71,0.071500,-1.000000,8.000000,779.000000,1.000000,179.000000,184.000000,1.000000,0.623850,0.274595,2518.000000,0.929941,0.966667,0.400000,34.333333,3,-13,164.000000
3,28,61,0.067304,-1.000000,8.000000,849.000000,1.000000,184.000000,196.000000,1.000000,0.592224,0.262095,2505.000000,0.894531,0.933333,0.733333,57.666667,3,-1,162.000000
4,11,24,0.165712,-1.000000,8.000000,102.000000,1.000000,250.000000,238.000000,3.000000,0.571137,0.088141,2569.000000,0.850497,0.000000,0.200000,54.333333,3,-4,195.000000


In [73]:
!pip install xgboost

In [74]:
#Separate X and y (explanatory variables and target variable in train set)
X_train = data_train.iloc[:,2:]
column_names = list(X_train) 
Y_train = data_train.iloc[:,0] 
del X_train['Retained_class']
X_train.head()

,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,Region,CRM_Segment,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.570244,0.119431,2708.000000,0.888166,0.000000,0.000000,31.333333,3,2,190.000000
1,other,air,East Coast,K,Middle,K->Middle,C,other,other,other,...,0.510360,0.190362,1250.605592,1.000000,0.000000,0.000000,0.000000,0,1,199.424613
2,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.623850,0.274595,2518.000000,0.929941,0.966667,0.400000,34.333333,3,-13,164.000000
3,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.592224,0.262095,2505.000000,0.894531,0.933333,0.733333,57.666667,3,-1,162.000000
4,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.571137,0.088141,2569.000000,0.850497,0.000000,0.200000,54.333333,3,-4,195.000000


In [75]:
Y_train.head()

0    0
1    1
2    1
3    1
4    0
Name: Retained, dtype: int64

In [76]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train_num) 

In [77]:
X_scaled = scaler.transform(X_train_num)
X_scaled = pd.DataFrame(X_scaled,columns=X_train_num.columns)

In [78]:
X_scaled.head()

,Days,Total_Pax,FPP_to_School_enrollment,MDR_Low_Grade,MDR_High_Grade,Total_School_Enrollment,NumberOfMeetingswithParents,DifferenceTraveltoFirstMeeting,DifferenceTraveltoLastMeeting,FRP_Cancelled,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,0.413949,-0.405257,1.316858,-1.392766,-0.224008,-1.214154,-0.152846,-0.248905,0.339865,-0.100863,0.013401,-0.561366,1.666322,-0.232089,-0.20787,-1.297590,-1.234349,-0.210760,0.373008,-0.108384
1,-0.340363,-0.843729,0.504914,-0.474224,-0.428943,-0.471839,-0.132563,0.030653,0.091888,-0.060880,-0.244007,-0.121039,-0.512006,1.762067,-0.20787,-1.297590,-3.429907,-11.877256,0.201655,0.082692
2,2.676883,1.027084,0.032444,-1.392766,-0.224008,0.365039,-0.152846,-1.019153,-0.835098,-0.626943,0.243827,0.401875,1.382334,0.512822,3.19469,0.199647,-1.024136,-0.210760,-2.197294,-0.635510
3,1.922572,0.734769,-0.020449,-1.392766,-0.224008,0.528324,-0.152846,-0.953878,-0.600105,-0.626943,0.107883,0.324272,1.362904,-0.118593,3.07736,1.447345,0.610853,-0.210760,-0.141052,-0.676058
4,-0.642088,-0.346794,1.219898,-1.392766,-0.224008,-1.214154,-0.152846,-0.092244,0.222369,-0.100863,0.017239,-0.755614,1.458563,-0.903789,-0.20787,-0.548972,0.377283,-0.210760,-0.655113,-0.007013


In [79]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train_num)

In [80]:
X_scaled = scaler.transform(X_train_num)
X_scaled = pd.DataFrame(X_scaled,columns=X_train_num.columns)

In [81]:
X_scaled.head()

,Days,Total_Pax,FPP_to_School_enrollment,MDR_Low_Grade,MDR_High_Grade,Total_School_Enrollment,NumberOfMeetingswithParents,DifferenceTraveltoFirstMeeting,DifferenceTraveltoLastMeeting,FRP_Cancelled,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,0.413949,-0.405257,1.316858,-1.392766,-0.224008,-1.214154,-0.152846,-0.248905,0.339865,-0.100863,0.013401,-0.561366,1.666322,-0.232089,-0.20787,-1.297590,-1.234349,-0.210760,0.373008,-0.108384
1,-0.340363,-0.843729,0.504914,-0.474224,-0.428943,-0.471839,-0.132563,0.030653,0.091888,-0.060880,-0.244007,-0.121039,-0.512006,1.762067,-0.20787,-1.297590,-3.429907,-11.877256,0.201655,0.082692
2,2.676883,1.027084,0.032444,-1.392766,-0.224008,0.365039,-0.152846,-1.019153,-0.835098,-0.626943,0.243827,0.401875,1.382334,0.512822,3.19469,0.199647,-1.024136,-0.210760,-2.197294,-0.635510
3,1.922572,0.734769,-0.020449,-1.392766,-0.224008,0.528324,-0.152846,-0.953878,-0.600105,-0.626943,0.107883,0.324272,1.362904,-0.118593,3.07736,1.447345,0.610853,-0.210760,-0.141052,-0.676058
4,-0.642088,-0.346794,1.219898,-1.392766,-0.224008,-1.214154,-0.152846,-0.092244,0.222369,-0.100863,0.017239,-0.755614,1.458563,-0.903789,-0.20787,-0.548972,0.377283,-0.210760,-0.655113,-0.007013


In [82]:
X_train_OK=X_train.copy()
X_train_OK.loc[:,X_train_num.columns]=X_scaled
X_train_OK.head()

,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,Region,CRM_Segment,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.013401,-0.561366,1.666322,-0.232089,-0.20787,-1.297590,-1.234349,-0.210760,0.373008,-0.108384
1,other,air,East Coast,K,Middle,K->Middle,C,other,other,other,...,-0.244007,-0.121039,-0.512006,1.762067,-0.20787,-1.297590,-3.429907,-11.877256,0.201655,0.082692
2,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.243827,0.401875,1.382334,0.512822,3.19469,0.199647,-1.024136,-0.210760,-2.197294,-0.635510
3,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,5,...,0.107883,0.324272,1.362904,-0.118593,3.07736,1.447345,0.610853,-0.210760,-0.141052,-0.676058
4,other,air,East Coast,PK,Middle,PK->Middle,C,other,Northern California,9,...,0.017239,-0.755614,1.458563,-0.903789,-0.20787,-0.548972,0.377283,-0.210760,-0.655113,-0.007013


In [143]:
X_train_OK=pd.get_dummies(X_train_OK,columns=X_train_OK.columns[:31])

In [144]:
#Separate X and y (explanatory variables and target variable in test set)
X_test = data_test.iloc[:,2:]
column_names = list(X_test) 
Y_test = data_test.iloc[:,0] 
del X_test['Retained_class']
X_test.head()

,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,Region,CRM_Segment,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,other,air,East Coast,K,High,K->High,C,other,Southern California,10,...,0.345746,0.103222,2167.0,0.991415,0.766667,0.829167,62.000000,3,-2,320.0
1,other,air,East Coast,PK,Middle,PK->Middle,C,other,Southern California,11,...,0.404058,0.104028,2148.0,0.915161,1.000000,0.533333,44.666667,3,4,213.0
2,other,air,East Coast,K,Middle,K->Middle,C,other,Pacific Northwest,1,...,0.517738,0.000000,1989.0,0.866585,1.000000,1.000000,23.000000,3,1,139.0
3,other,air,East Coast,Middle,Middle,Middle->Middle,C,other,Southern California,5,...,0.721320,0.381479,2388.0,0.887128,0.000000,0.000000,50.000000,3,-12,399.0
4,other,air,East Coast,K,Middle,K->Middle,C,other,Southern California,5,...,0.492463,0.168508,2360.0,0.957977,0.700000,0.800000,23.000000,4,-4,207.0


In [113]:
X_test_num = X_test.iloc[:,31:]
X_test_num.head()

,Days,Total_Pax,FPP_to_School_enrollment,MDR_Low_Grade,MDR_High_Grade,Total_School_Enrollment,NumberOfMeetingswithParents,DifferenceTraveltoFirstMeeting,DifferenceTraveltoLastMeeting,FRP_Cancelled,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,24,29,0.126429,0.0,12.0,196.0,2.0,332.0,215.0,1.0,0.345746,0.103222,2167.0,0.991415,0.766667,0.829167,62.000000,3,-2,320.0
1,24,25,0.142807,-1.0,8.0,158.0,1.0,265.0,254.0,3.0,0.404058,0.104028,2148.0,0.915161,1.000000,0.533333,44.666667,3,4,213.0
2,13,7,0.108986,0.0,8.0,55.0,1.0,180.0,189.0,0.0,0.517738,0.000000,1989.0,0.866585,1.000000,1.000000,23.000000,3,1,139.0
3,30,69,0.246745,7.0,8.0,214.0,1.0,280.0,281.0,9.0,0.721320,0.381479,2388.0,0.887128,0.000000,0.000000,50.000000,3,-12,399.0
4,24,26,0.113636,0.0,8.0,217.0,2.0,372.0,250.0,7.0,0.492463,0.168508,2360.0,0.957977,0.700000,0.800000,23.000000,4,-4,207.0


In [114]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train_num) 

In [115]:
X_scaled_test = scaler.transform(X_test_num)
X_scaled_test = pd.DataFrame(X_scaled_test,columns=X_test_num.columns)
X_scaled_test.head()

,Days,Total_Pax,FPP_to_School_enrollment,MDR_Low_Grade,MDR_High_Grade,Total_School_Enrollment,NumberOfMeetingswithParents,DifferenceTraveltoFirstMeeting,DifferenceTraveltoLastMeeting,FRP_Cancelled,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,1.319123,-0.200637,0.724774,-1.099987,2.051241,-0.994887,1.482740,0.978271,-0.228033,-0.626943,-0.951598,-0.661994,0.857704,1.608977,2.490712,1.806058,0.914494,-0.210760,-0.312406,2.527248
1,1.319123,-0.317563,0.931198,-1.392766,-0.224008,-1.083527,-0.152846,0.103582,0.535693,-0.100863,-0.700943,-0.656987,0.829305,0.249267,3.312020,0.698726,-0.300069,-0.210760,0.715715,0.357920
2,-0.340363,-0.843729,0.504914,-1.099987,-0.224008,-1.323788,-0.152846,-1.006098,-0.737184,-0.889982,-0.212293,-1.302783,0.591652,-0.616906,3.312020,2.445503,-1.818274,-0.210760,0.201655,-1.142362
3,2.224297,0.968621,2.241234,0.949469,-0.224008,-0.952899,-0.152846,0.299408,1.064426,1.477376,0.662799,1.065399,1.188027,-0.250597,-0.207870,-1.297590,0.073643,-0.210760,-2.025940,4.128901
4,1.319123,-0.288331,0.563528,-1.099987,-0.224008,-0.945901,1.482740,1.500473,0.457362,0.951296,-0.320936,-0.256706,1.146176,1.012730,2.256053,1.696884,-1.818274,3.678071,-0.655113,0.236276


In [164]:
X_test_OK=X_test.copy()
X_test_OK.loc[:,X_test_num.columns]=X_scaled_test
X_test_OK.head()

,Program_Code,Travel_Type,SPR_Product_Type,GroupGradeTypeLow,GroupGradeTypeHigh,GroupGradeType,MajorProgramCode,Poverty_Code,Region,CRM_Segment,...,FRP_Take_up_percent_,EZ_Pay_Take_Up_Rate,SPR_Group_Revenue,FPP_to_PAX,pol_mean,sub_mean,twl_mean,numTweets,totPaxDiff,depositToDeparture
0,other,air,East Coast,K,High,K->High,C,other,Southern California,10,...,-0.951598,-0.661994,0.857704,1.608977,2.490712,1.806058,0.914494,-0.210760,-0.312406,2.527248
1,other,air,East Coast,PK,Middle,PK->Middle,C,other,Southern California,11,...,-0.700943,-0.656987,0.829305,0.249267,3.312020,0.698726,-0.300069,-0.210760,0.715715,0.357920
2,other,air,East Coast,K,Middle,K->Middle,C,other,Pacific Northwest,1,...,-0.212293,-1.302783,0.591652,-0.616906,3.312020,2.445503,-1.818274,-0.210760,0.201655,-1.142362
3,other,air,East Coast,Middle,Middle,Middle->Middle,C,other,Southern California,5,...,0.662799,1.065399,1.188027,-0.250597,-0.207870,-1.297590,0.073643,-0.210760,-2.025940,4.128901
4,other,air,East Coast,K,Middle,K->Middle,C,other,Southern California,5,...,-0.320936,-0.256706,1.146176,1.012730,2.256053,1.696884,-1.818274,3.678071,-0.655113,0.236276


In [165]:
X_test_OK=pd.get_dummies(X_test_OK,columns=X_test_OK.columns[:31])

In [166]:
missing_cols = set( X_train_OK.columns ) - set( X_test_OK.columns )
#Add a missing column in test set with default value equal to 0
for c in missing_cols:
    X_test_OK[c] = 0
# Ensure the order of column in the test set is in the same order than in train set
X_test_OK = X_test_OK[X_train_OK.columns]

In [168]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [169]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

In [175]:
train_df = pd.read_csv('train.csv', dtype={'id': np.int32, 'Retained': np.int8})
Y = Y_train
X = X_train_OK
test_df = pd.read_csv('test.csv', dtype={'id': np.object})
Y = Y_test
X = X_test_OK

In [176]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [178]:
folds = 3
param_comb = 2

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   6 | elapsed:  5.4min remaining: 10.8min
[Parallel(n_jobs=4)]: Done   6 out of   6 | elapsed:  8.2min finished


[15:17:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:17:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 13 minutes and 11.19 seconds.


In [180]:
print('\n All results:')
print(random_search.cv_results_)
print('\n Best estimator:')
print(random_search.best_estimator_)
print('\n Best normalized gini score for %d-fold search with %d parameter combinations:' % (folds, param_comb))
print(random_search.best_score_ * 2 - 1)
print('\n Best hyperparameters:')
print(random_search.best_params_)
results = pd.DataFrame(random_search.cv_results_)
results.to_csv('xgb-random-grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([324.12434769, 204.2209506 ]), 'std_fit_time': array([ 2.08980219, 33.82679007]), 'mean_score_time': array([1.01719562, 0.98429362]), 'std_score_time': array([0.01998139, 0.18326253]), 'param_subsample': masked_array(data=[1.0, 0.6],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[5, 10],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[4, 4],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[2, 5],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'param_colsample_bytree': masked_array(data=[1.0, 1.0],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'subsample': 1.0, 'min_child_weight': 5, 'max_depth': 4, 'gamma': 2, 'colsample_bytree': 1.0}, {'sub

In [183]:
y_test = random_search.predict_proba(X_test_OK)
y_test

array([[0.18503195, 0.81496805],
       [0.14731902, 0.852681  ],
       [0.32113975, 0.67886025],
       ...,
       [0.37375188, 0.6262481 ],
       [0.6151217 , 0.38487825],
       [0.5256189 , 0.4743811 ]], dtype=float32)

In [220]:
import xgboost as xgb
from xgboost import plot_importance

In [221]:
xgb1 = xgb.XGBRegressor(objective = 'reg:squarederror')

In [232]:
xgb1.fit(X_train_OK, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [233]:
xgb = xgb1.fit (X_train_OK , Y_train)

In [234]:
#EVALUATE THE PREDICTION (on the y_test)
from sklearn.metrics import f1_score
print('***RESULTS ON TEST SET***')
print("f1_score: ", f1_score(Y_test, y_pred))

***RESULTS ON TEST SET***
f1_score:  0.8308977035490605
